In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import geograpy as gg
import pycountry as pc

In [38]:
df = pd.read_csv("output-#destination-5000-since-2019-10-28.csv")
df.head()

,id,username,text,retweets,favorites,replies,permalink,date,mentions,hashtags
0,1200509010174038016,FashionNourishd,"Watching a #JacksonChameleon #climb up a tree,...",0,0,0,https://twitter.com/FashionNourishd/status/120...,2019-11-29 20:17:07+00:00,NaN,#JacksonChameleon #climb #figureout #nextstep ...
1,1200503121920659456,CheapFlyME,#Vancouver #Canada To #London #UK $C438 Jan 15...,0,0,0,https://twitter.com/CheapFlyME/status/12005031...,2019-11-29 19:53:43+00:00,NaN,#Vancouver #Canada #London #UK #TravelTuesday ...
2,1200500471577956352,Sorothavy,#booking #flight #adventure #explore #destinat...,1,0,0,https://twitter.com/Sorothavy/status/120050047...,2019-11-29 19:43:11+00:00,NaN,#booking #flight #adventure #explore #destination
3,1200493123983405056,TravelGumbo,Today's featured #destination #travelblog. #Ro...,0,0,0,https://twitter.com/TravelGumbo/status/1200493...,2019-11-29 19:13:59+00:00,NaN,#destination #travelblog #Ron #Antique #Radio ...
4,1200474587294769154,KASAHotelParota,Welcome to #Paradise. It's really amazing to v...,0,0,0,https://twitter.com/KASAHotelParota/status/120...,2019-11-29 18:00:20+00:00,NaN,#Paradise #Tulum #vacation #beach #smallluxury...


In [39]:
tknzr = TweetTokenizer()
stop_words = set(stopwords.words('english'))

#Use these for output csv that have location
# df['location_filled'] = df['location'].fillna("")
# df['Location_Token'] = df['location_filled'].apply(lambda x: x.split(","))
# df['Tweet_Token'] = df['tweet'].apply(lambda x: tknzr.tokenize(x))
df['Tweet_Token'] = df['text'].apply(lambda x: tknzr.tokenize(x))
df['Tweet_Token_RemoveStop'] = df['Tweet_Token'].apply(lambda x: [w for w in x if not w in stop_words])

In [40]:
#List of Location from pycountry.country & subdivision
location_list = []
for country in list(pc.countries):
    print(country)
    location_list.append(country.alpha_2.lower())
    location_list.append(country.alpha_3.lower())
    location_list.append(country.name.lower())
    try:
        location_list.append(country.common_name.lower())
    except:
        None
for city in list(pc.subdivisions):
    location_list.append(city.name.lower())
    print(city)
    try:
        location_list.append(city.city.code[-2:].lower())
    except:
        None
location_list.append('philadelphia')
location_list.append('fl')
location_list.append('nyc')
Remove_list = ['and', 'as', 'at', 'can', 'in', 'is','me', 'my','it','us','to','by','most','be','so']
for remove_word in Remove_list:
    location_list.remove(remove_word)

Country(alpha_2='AW', alpha_3='ABW', name='Aruba', numeric='533')
Country(alpha_2='AF', alpha_3='AFG', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan')
Country(alpha_2='AO', alpha_3='AGO', name='Angola', numeric='024', official_name='Republic of Angola')
Country(alpha_2='AI', alpha_3='AIA', name='Anguilla', numeric='660')
Country(alpha_2='AX', alpha_3='ALA', name='Åland Islands', numeric='248')
Country(alpha_2='AL', alpha_3='ALB', name='Albania', numeric='008', official_name='Republic of Albania')
Country(alpha_2='AD', alpha_3='AND', name='Andorra', numeric='020', official_name='Principality of Andorra')
Country(alpha_2='AE', alpha_3='ARE', name='United Arab Emirates', numeric='784')
Country(alpha_2='AR', alpha_3='ARG', name='Argentina', numeric='032', official_name='Argentine Republic')
Country(alpha_2='AM', alpha_3='ARM', name='Armenia', numeric='051', official_name='Republic of Armenia')
Country(alpha_2='AS', alpha_3='ASM', name='American Samoa', n

Subdivision(code='GB-BDG', country_code='GB', name='Barking and Dagenham', parent='ENG', parent_code='GB-ENG', type='London borough')
Subdivision(code='GB-BEN', country_code='GB', name='Brent', parent='ENG', parent_code='GB-ENG', type='London borough')
Subdivision(code='GB-BEX', country_code='GB', name='Bexley', parent='ENG', parent_code='GB-ENG', type='London borough')
Subdivision(code='GB-BFS', country_code='GB', name='Belfast', parent='NIR', parent_code='GB-NIR', type='District')
Subdivision(code='GB-BGE', country_code='GB', name='Bridgend; Pen-y-bont ar Ogwr', parent='WLS', parent_code='GB-WLS', type='Unitary authority')
Subdivision(code='GB-BGW', country_code='GB', name='Blaenau Gwent', parent='WLS', parent_code='GB-WLS', type='Unitary authority')
Subdivision(code='GB-BIR', country_code='GB', name='Birmingham', parent='ENG', parent_code='GB-ENG', type='Metropolitan district')
Subdivision(code='GB-BKM', country_code='GB', name='Buckinghamshire', parent='ENG', parent_code='GB-ENG', 

Subdivision(code='MK-74', country_code='MK', name='Studeničani', parent_code=None, type='Municipality')
Subdivision(code='MK-75', country_code='MK', name='Tearce', parent_code=None, type='Municipality')
Subdivision(code='MK-76', country_code='MK', name='Tetovo', parent_code=None, type='Municipality')
Subdivision(code='MK-77', country_code='MK', name='Centar', parent_code=None, type='Municipality')
Subdivision(code='MK-78', country_code='MK', name='Centar Župa', parent_code=None, type='Municipality')
Subdivision(code='MK-79', country_code='MK', name='Čair', parent_code=None, type='Municipality')
Subdivision(code='MK-80', country_code='MK', name='Čaška', parent_code=None, type='Municipality')
Subdivision(code='MK-81', country_code='MK', name='Češinovo-Obleševo', parent_code=None, type='Municipality')
Subdivision(code='MK-82', country_code='MK', name='Čučer Sandevo', parent_code=None, type='Municipality')
Subdivision(code='MK-83', country_code='MK', name='Štip', parent_code=None, type='Mu

Subdivision(code='TR-69', country_code='TR', name='Bayburt', parent_code=None, type='Province')
Subdivision(code='TR-70', country_code='TR', name='Karaman', parent_code=None, type='Province')
Subdivision(code='TR-71', country_code='TR', name='Kırıkkale', parent_code=None, type='Province')
Subdivision(code='TR-72', country_code='TR', name='Batman', parent_code=None, type='Province')
Subdivision(code='TR-73', country_code='TR', name='Şırnak', parent_code=None, type='Province')
Subdivision(code='TR-74', country_code='TR', name='Bartın', parent_code=None, type='Province')
Subdivision(code='TR-75', country_code='TR', name='Ardahan', parent_code=None, type='Province')
Subdivision(code='TR-76', country_code='TR', name='Iğdır', parent_code=None, type='Province')
Subdivision(code='TR-77', country_code='TR', name='Yalova', parent_code=None, type='Province')
Subdivision(code='TR-78', country_code='TR', name='Karabük', parent_code=None, type='Province')
Subdivision(code='TR-79', country_code='TR',

In [41]:
def country_name(list):
    if list == []:
        return np.nan
    else:
        lower_list = []
        Location = []
        for token in list:
            lower_list.append(token.lower())
        for location in location_list:
            if location in lower_list:
                Location.append(location)
        if Location == []:
            for value in lower_list:
                fuzz_score_country = {}
                for country in location_list:
                    fuzz_score_country[country] = fuzz.ratio(value,country)
                for country in ['united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
                    fuzz_score_country[country] = fuzz.ratio(value,country)
                for country in ['the netherlands','england','u.k.','uk','uk ',' uk','uae','korea','south korea']:
                    fuzz_score_country[country] = fuzz.ratio(value,country)
            if max(fuzz_score_country, key=fuzz_score_country.get) in ['cascadia','united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
                Location.append(pc.countries.lookup('united states').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['england','u.k.','uk','uk ',' uk']:
                Location.append(pc.countries.lookup('united kingdom').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['uae']:
                Location.append(pc.countries.lookup('United Arab Emirates').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['korea','south korea']:
                Location.append(pc.countries.lookup('KR').alpha_3)
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['the netherlands']:
                Location.append(pc.countries.lookup('netherlands').alpha_3)
            elif fuzz_score_country[max(fuzz_score_country, key=fuzz_score_country.get)] <70:
                if Location == []:
                    return np.nan
                else:
                    return Location
            else:
                Location.append(max(fuzz_score_country, key=fuzz_score_country.get))
                return Location
        else:
            return Location
                
#                 fuzz_score_city = {}
#                 for city in city_list:
#                     fuzz_score_city[city] = fuzz.ratio(value,city)
#                 if fuzz_score_city[max(fuzz_score_city, key=fuzz_score_city.get)] <90:
#                     return np.nan
#                 else:
#                     return pc.countries.lookup(pc.subdivisions.lookup(max(fuzz_score_city, key=fuzz_score_city.get)).country_code).alpha_3
#             return pc.countries.lookup(max(fuzz_score_country, key=fuzz_score_country.get)).alpha_3
#         print(fuzz_score_country[max(fuzz_score_country)])
        


In [42]:
#An example of a good tweet
# country_name(df['Tweet_Token'][984])

In [43]:
#Apply country_name on locations
# df['Location_Found'] = df['Location_Token'].apply(lambda x: country_name(x))
# df['Location_Found']

In [44]:
#Apply country_name on tweets
df['Tweet_Location_Found'] = df['Tweet_Token_RemoveStop'].apply(lambda x: country_name(x))
df['Tweet_Location_Found']

0                           NaN
1                           NaN
2                           NaN
3                          None
4                           NaN
5                 [switzerland]
6                           NaN
7                 [philippines]
8                           NaN
9                          [de]
10                         [rs]
11                          NaN
12                          NaN
13                          NaN
14                 [de, et, la]
15               [south, south]
16                 [de, et, la]
17                          NaN
18                        [cod]
19                          NaN
20                          NaN
21                          NaN
22                          NaN
23                          NaN
24                          NaN
25      [indonesia, pool, bali]
26                     [bhutan]
27                          NaN
28                          NaN
29                          NaN
                 ...           
4970    

In [16]:
df.to_csv('output_combined_location.csv',index=False)
pd.read_csv("output_combined_location.csv")

,id,username,text,retweets,favorites,replies,permalink,date,mentions,hashtags,Tweet_Token,Tweet_Location_Found
0,1200516560160481280,vanderfullife,Pan Pacific Airlines adds Clark – Muan route f...,0,0,0,https://twitter.com/vanderfullife/status/12005...,2019-11-29 20:47:07+00:00,NaN,#news #today #travel #vacation #cruise #holida...,"['Pan', 'Pacific', 'Airlines', 'adds', 'Clark'...",['pan']
1,1200516487389122560,sweetsteph0914,Today I was presented w/ an #eviction notice (...,0,0,0,https://twitter.com/sweetsteph0914/status/1200...,2019-11-29 20:46:49+00:00,@Twitter,#eviction #holidays,"['Today', 'I', 'was', 'presented', 'w', '/', '...","['at', 'can', 'me', 'my', 'to']"
2,1200516453612568576,PhilMorseRE,Earning extra money during the #holidays can m...,0,0,0,https://twitter.com/PhilMorseRE/status/1200516...,2019-11-29 20:46:41+00:00,NaN,#holidays #creditcards,"['Earning', 'extra', 'money', 'during', 'the',...","['can', 'it', 'to']"
3,1200516306967154688,HollyNorth,"Oh the weather outside is frightful, and our s...",0,0,0,https://twitter.com/HollyNorth/status/12005163...,2019-11-29 20:46:06+00:00,NaN,#Holidays #HollyNorth #HollyNorthProductionSup...,"['Oh', 'the', 'weather', 'outside', 'is', 'fri...","['and', 'is', 'it', 'so']"
4,1200516102205239296,LissaandTiniRay,Hanging with some of my favs @CranbrookLiquo1 ...,0,0,0,https://twitter.com/LissaandTiniRay/status/120...,2019-11-29 20:45:17+00:00,@CranbrookLiquo1,#g4tequila #liquorstores #tastings #lovethiscr...,"['Hanging', 'with', 'some', 'of', 'my', 'favs'...","['and', 'my']"
5,1200516085835014150,TheDarianH,New single out now! Stream for free on all str...,0,0,0,https://twitter.com/TheDarianH/status/12005160...,2019-11-29 20:45:14+00:00,NaN,#christmas #friday #holidays #newmusic #new #m...,"['New', 'single', 'out', 'now', '!', 'Stream',...",['in']
6,1200516039748046849,GallagherWayChi,There's nothing quite like DIY-ing during the ...,0,0,0,https://twitter.com/GallagherWayChi/status/120...,2019-11-29 20:45:03+00:00,NaN,#holidays #WinterlandatGallagherWay,"[""There's"", 'nothing', 'quite', 'like', 'DIY-i...",['at']
7,1200516028201082880,touchstonewdc,The #holidays are here! Rent our #gallery #spa...,0,0,0,https://twitter.com/touchstonewdc/status/12005...,2019-11-29 20:45:00+00:00,NaN,#holidays #gallery #space #party #event #dc #r...,"['The', '#holidays', 'are', 'here', '!', 'Rent...",['are']
8,1200515578215174145,neilwalkerwrote,This #action-packed #crime #thriller is set ov...,0,0,0,https://twitter.com/neilwalkerwrote/status/120...,2019-11-29 20:43:13+00:00,NaN,#action #crime #thriller #Christmas #newyear #...,"['This', '#action-packed', '#crime', '#thrille...","['by', 'is']"
9,1200515547089264641,1423PetsmartSM,Check out our great black friday sales! #petsm...,0,0,0,https://twitter.com/1423PetsmartSM/status/1200...,2019-11-29 20:43:05+00:00,NaN,#petsmart #sanmarcos #fortheloveofpets #holida...,"['Check', 'out', 'our', 'great', 'black', 'fri...",NaN


In [45]:
#Run this to get all valid locations, remove empty location records
df_copy = df.copy()
df_copy = df_copy.dropna(subset = ['Tweet_Location_Found'])
print(df_copy['Tweet_Location_Found'])

5                  [switzerland]
7                  [philippines]
9                           [de]
10                          [rs]
14                  [de, et, la]
15                [south, south]
16                  [de, et, la]
18                         [cod]
25       [indonesia, pool, bali]
26                      [bhutan]
36              [southern, gulf]
39                          [so]
44                 [switzerland]
45                    [thailand]
47                [aruba, aruba]
52                 [uttarakhand]
55                     [morelos]
56                          [ms]
57                [south, south]
61                   [islamabad]
64                      [de, la]
66                        [york]
67                          [dm]
68                    [maldives]
73                      [canada]
74                          [la]
76                      [greece]
81                   [singapore]
83                        [bali]
84                [south, south]
          

In [3]:
#Run this to get all valid locations, remove empty location records
df_copy = pd.read_csv("output_combined_location.csv")
df_copy = df_copy.dropna(subset = ['Tweet_Location_Found'])
print(df_copy['Tweet_Location_Found'])

0                                                  ['pan']
1                          ['at', 'can', 'me', 'my', 'to']
2                                      ['can', 'it', 'to']
3                                ['and', 'is', 'it', 'so']
4                                            ['and', 'my']
5                                                   ['in']
6                                                   ['at']
7                                                  ['are']
8                                             ['by', 'is']
10                                      ['at', 'be', 'in']
11                                           ['and', 'so']
12                                           ['and', 'as']
13                                           ['che', 'il']
14                            ['in', 'to', 'turkey', 'us']
15                                           ['and', 'to']
16                                            ['in', 'it']
17                                                  ['is

In [195]:
#Testing
df['Location_Token'][980:990]

980                 [Alto,  TN]
981        [Thunder Bay/Ottawa]
982                   [Cardiff]
983                       [BCK]
984             [Oslo,  Norway]
985                          []
986                          []
987    [Altrincham,  South Mcr]
988                          []
989           [Cincinnati,  OH]
Name: Location_Token, dtype: object